In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy import stats
from scipy.stats import linregress
from config import weather_api_key
#from config import g_key
from citipy import citipy

In [8]:
#set "cit center" for Houston
Houston_lat = 29.756097853207656
Houston_lng = -95.3669907972107


#calculate greater Houston area by expanding lat and lng
lat = np.random.uniform(Houston_lat-1, Houston_lat+2, size=6500)
lng = np.random.uniform(Houston_lng-5, Houston_lng+2, size=6500)
city = []

#find nearest city using lat and lng
for x in range(len(lat)):
    city.append(citipy.nearest_city(lat[x], lng[x]).city_name)


In [9]:
#turn cities into df
df_city = pd.DataFrame({
    "City":city,
    "lat":lat,
    "lng":lng,
})

In [10]:
df_city.head(20)

,City,lat,lng
0,leander,30.746499,-98.082715
1,port arthur,29.179217,-93.741184
2,victoria,28.854315,-97.404748
3,huntsville,30.590083,-95.504712
4,port arthur,28.925866,-93.826561
5,port arthur,29.288621,-93.843978
6,lufkin,31.060962,-95.088400
7,uvalde,29.895818,-100.079630
8,victoria,28.765316,-96.954376
9,brownwood,31.639487,-98.736397


In [11]:
#check length of cities
len(list(df_city["City"].unique()))

113

In [12]:
cities = list(df_city["City"].unique())
target_cities=[]

for i in cities:
    c=str(i).replace(" ","+")
    target_cities.append(c)

target_cities

['leander',
 'port+arthur',
 'victoria',
 'huntsville',
 'lufkin',
 'uvalde',
 'brownwood',
 'corsicana',
 'lockhart',
 'hondo',
 'copperas+cove',
 'natchitoches',
 'katy',
 'seguin',
 'bay+city',
 'wharton',
 'kerrville',
 'baytown',
 'brenham',
 'pleasanton',
 'sulphur',
 'orange',
 'belton',
 'san+angelo',
 'gatesville',
 'galveston',
 'jacksonville',
 'bryan',
 'leon+valley',
 'lakeway',
 'live+oak',
 'new+braunfels',
 'el+campo',
 'austin',
 'taylor',
 'friendswood',
 'killeen',
 'santa+fe',
 'liberty',
 'lumberton',
 'conroe',
 'freeport',
 'dickinson',
 'palestine',
 'rosenberg',
 'beeville',
 'lake+jackson',
 'nacogdoches',
 'fredericksburg',
 'tomball',
 'jacinto+city',
 'robinson',
 'stephenville',
 'college+station',
 'bellmead',
 'mission+bend',
 'woodway',
 'deer+park',
 'houston',
 'angleton',
 'spring',
 'georgetown',
 'pflugerville',
 'temple',
 'missouri+city',
 'nederland',
 'kyle',
 'vidor',
 'port+lavaca',
 'kirby',
 'hewitt',
 'pearland',
 'san+antonio',
 'canyon+l

In [13]:
#set function to get today and previous 4 days' weather data
def unix_time():
    today = datetime.date.today()
    days = [1,2,3,4]
    times = [int(time.mktime(today.timetuple()))]
    for day in days:
        x=today - datetime.timedelta(days=day)
        unixtime = int(time.mktime(x.timetuple()))
        times.append(unixtime)
    return (times)

In [ ]:
 base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
temp=[]
wind_speed=[]
pressure=[]
city_name=[]

for city in target_cities[:4]:
    query_url=f"{base_current_url}appid={weather_api_key}&q={city},US&units=imperial"
    req = requests.get(query_url)
    response1=req.json()
    latitude=response1["coord"]["lat"]
    longitude=response1["coord"]["lon"]

    for x in unix_time():
        historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
        hist_json = requests.get(historical_url).json()
        temp.append(hist_json["current"]["temp"])
        pressure.append(hist_json["current"]["pressure"])
        wind_speed.append(hist_json["current"]["wind_speed"])
        city_name.append(response1['name'])
        print(historical_url)

In [33]:
#set base url and define function to find OpenWeather cities' based on city name from nearest place API and feed back into
#OpenWeather one call API to ensure proper lat and lng

def find_location(target_cities):
    unix_time()
    base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
    temp=[]
    wind_speed=[]
    pressure=[]
    city_name=[]
    country_code=[]
    for city in target_cities[:5]:
        try: 
            query_url=f"{base_current_url}appid={weather_api_key}&q={city},Texas,USA&units=imperial"
            req = requests.get(query_url)
            response1=req.json()
            latitude=response1["coord"]["lat"]
            longitude=response1["coord"]["lon"]
#             print(response1)

        except:
            print("City not found. Skipping...")
            pass
        if response1["sys"]["country"]=="US":
            for x in unix_time():
                historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
                hist_json = requests.get(historical_url).json()
                temp.append(hist_json["current"]["temp"])
                pressure.append(hist_json["current"]["pressure"])
                wind_speed.append(hist_json["current"]["wind_speed"])
                city_name.append(response1['name'])
                country_code.append(response1["sys"]["country"])
    d=dict()
    d["City Name"] = city_name
    d["Country"] = country_code
    d["Temperature"] = temp
    d["Wind_Speed"] = wind_speed
    d["Pressure"] = pressure    
    return(d)
        #print ("--"*30)

In [34]:
df2 =pd.DataFrame(find_location(target_cities))

{'coord': {'lon': -97.8444, 'lat': 30.5586}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 92.32, 'feels_like': 100.36, 'temp_min': 90.01, 'temp_max': 94.19, 'pressure': 1001, 'humidity': 52}, 'visibility': 10000, 'wind': {'speed': 7.14, 'deg': 152, 'gust': 13.04}, 'clouds': {'all': 69}, 'dt': 1630455337, 'sys': {'type': 2, 'id': 2009793, 'country': 'US', 'sunrise': 1630411663, 'sunset': 1630457755}, 'timezone': -18000, 'id': 4705708, 'name': 'Leander', 'cod': 200}
{'coord': {'lon': -93.9288, 'lat': 29.8988}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 84.87, 'feels_like': 97.47, 'temp_min': 82.26, 'temp_max': 86.02, 'pressure': 1007, 'humidity': 89}, 'visibility': 10000, 'wind': {'speed': 5.75, 'deg': 190}, 'clouds': {'all': 1}, 'dt': 1630455128, 'sys': {'type': 1, 'id': 3495, 'country': 'US', 'sunrise': 1630410756, 'sunset': 

In [35]:
df2

,City Name,Country,Temperature,Wind_Speed,Pressure
0,Leander,US,78.82,1.99,1005
1,Leander,US,79.05,1.99,1008
2,Leander,US,74.95,1.01,1010
3,Leander,US,77.36,1.99,1010
4,Leander,US,78.57,0.00,1010
5,Port Arthur,US,78.60,4.61,1008
6,Port Arthur,US,78.60,10.36,1009
7,Port Arthur,US,76.21,5.75,1014
8,Port Arthur,US,77.77,4.61,1014
9,Port Arthur,US,79.34,11.50,1014


In [20]:
df[df["Country"]=="US"]

,City Name,Country,Temperature,Wind_Speed,Pressure
0,Leander,US,78.82,1.99,1005
1,Leander,US,79.05,1.99,1008
2,Leander,US,74.95,1.01,1010
3,Leander,US,77.36,1.99,1010
4,Leander,US,78.57,0.00,1010
5,Port Arthur,US,78.60,4.61,1008
6,Port Arthur,US,78.60,10.36,1009
7,Port Arthur,US,76.21,5.75,1014
8,Port Arthur,US,77.77,4.61,1014
9,Port Arthur,US,79.34,11.50,1014
